# Polars

In [2]:
import polars as pl

### Load Data

In [3]:
DATA_DIR = "../data/dota"

In [4]:
pl_match = pl.scan_csv(f"{DATA_DIR}/match.csv")
pl_cluster_regions = pl.scan_csv(f"{DATA_DIR}/cluster_regions.csv")
pl_purchase_log = pl.scan_csv(f"{DATA_DIR}/purchase_log.csv")
pl_item_id_names = pl.scan_csv(f"{DATA_DIR}/item_ids.csv")
pl_players = pl.scan_csv(f"{DATA_DIR}/players.csv")

In [5]:
# add region names to match data

match_with_region = pl_match.join(pl_cluster_regions, how="left", on="cluster").drop("cluster")

In [8]:
# add item names to item purchases

purchases_with_item_names = pl_purchase_log.join(pl_item_id_names, how="left", on="item_id").drop("item_id").groupby(["match_id", "player_slot", "item_name"]).agg(pl.col("time").list().keep_name()).groupby(["match_id", "player_slot"]).agg(
    [
        pl.apply(
            [pl.col("item_name"), pl.col("time")],
            lambda s: dict(zip(s[0], s[1].to_list())),
        ).alias("purchases")
    ]
)

purchases_with_item_names.head(15)

In [9]:
pl_purchase_log.head(3)

In [13]:
purchases_with_item_names.fetch()

match_id,player_slot,purchases
i64,i64,object
9416,0,"{'ancient_janggo': [683], 'manta': [1459], 'blink': [1623], 'belt_of_strength': [306], 'boots': [117], 'gauntlets': [500], 'ultimate_orb': [1436], 'tango': [-65], 'quelling_blade': [22], 'gloves': [306], 'ring_of_basilius': [986], 'boots_of_elves': [1222], 'headdress': [937], 'branches': [-67, -67, 937], 'clarity': [502], 'yasha': [1222], 'ring_of_protection': [944], 'lifesteal': [946], 'magic_wand': [337], 'power_treads': [337], 'ring_of_regen': [937], 'bracer': [500], 'blade_of_alacrity': [1222], 'tpscroll': [375, 445, 1067, 1067], 'circlet': [-70, -69, 500], 'robe': [649, 652, 671], 'magic_stick': [306], 'sobi_mask': [986], 'vladmir': [1012], 'flask': [-65, 374, 503, 1001]}"
6614,132,"{'gloves': [500], 'power_treads': [602], 'ward_observer': [-70, 772, 772], 'blade_of_alacrity': [1084, 1153], 'slippers': [-62, -62], 'robe': [1208], 'circlet': [-57], 'boots': [104], 'belt_of_strength': [578], 'wraith_band': [401], 'orb_of_venom': [206], 'diffusal_blade': [1510], 'tango': [-64], 'ward_sentry': [855, 855, 856]}"
11156,1,"{'courier': [-84], 'medallion_of_courage': [970], 'magic_stick': [-81], 'boots': [50], 'arcane_boots': [741], 'shadow_amulet': [1134], 'energy_booster': [741], 'glimmer_cape': [1150], 'branches': [-75, 915], 'clarity': [-79, -77], 'tango': [-82], 'cloak': [1025], 'ward_observer': [1002], 'tpscroll': [189, 233, 501, 601], 'flying_courier': [188], 'magic_wand': [1000], 'sobi_mask': [969], 'chainmail': [906], 'circlet': [915]}"
49362,3,"{'bracer': [312], 'power_treads': [1174], 'gloves': [507, 1135], 'tango': [-77], 'gauntlets': [-79], 'yasha': [2004], 'helm_of_the_dominator': [2061], 'branches': [-68, -67], 'blades_of_attack': [249], 'boots_of_elves': [1877], 'ancient_janggo': [1380], 'boots': [143], 'helm_of_iron_will': [836, 2061], 'belt_of_strength': [1073, 1600], 'robe': [1312], 'circlet': [-78], 'ogre_axe': [1494], 'sange_and_yasha': [2004], 'lifesteal': [2058], 'armlet': [952], 'blade_of_alacrity': [1877], 'sange': [1919], 'tpscroll': [541, 673, 803, 1180, 1181]}"
44874,0,"{'vanguard': [1206], 'chainmail': [1258, 1684], 'ogre_axe': [2220], 'black_king_bar': [2220], 'crimson_guard': [1735], 'magic_stick': [293], 'tranquil_boots': [289], 'circlet': [293], 'robe': [1452], 'blade_mail': [1452], 'mithril_hammer': [2220], 'smoke_of_deceit': [1973], 'stout_shield': [-83], 'tpscroll': [289, 391, 558, 633, 795, 898, 1011, 1281, 1454, 2159], 'vitality_booster': [1119], 'ring_of_regen': [89], 'magic_wand': [293], 'ring_of_health': [1179], 'ring_of_protection': [-83], 'buckler': [1684], 'branches': [-81, -81, 1684], 'tango': [-82], 'boots': [224], 'broadsword': [1452], 'blink': [783]}"
3000,132,"{'bfury': [1347], 'assault': [2082], 'broadsword': [1325], 'void_stone': [920], 'boots': [466], 'belt_of_strength': [1555], 'pers': [941], 'branches': [1708], 'headdress': [1708], 'boots_of_elves': [622, 1439], 'yasha': [1500], 'claymore': [1124], 'tango': [-41], 'platemail': [1973], 'quelling_blade': [-41], 'ring_of_basilius': [1708], 'hyperstone': [1900], 'gloves': [747], 'demon_edge': [2168], 'monkey_king_bar': [2200], 'power_treads': [807], 'ring_of_regen': [1708], 'poor_mans_shield': [87], 'blade_of_alacrity': [1399, 1424], 'tpscroll': [354, 469, 580, 674, 804, 877, 1012, 1136, 1186, 1397, 1577, 1884], 'sange': [1574], 'stout_shield': [-42], 'ring_of_health': [213], 'lifesteal': [1709], 'slippers': [87, 87], 'ring_of_protection': [1708], 'sange_and_yasha': [1576], 'vladmir': [1735], 'ogre_axe': [1554], 'sobi_mask': [1708], 'javelin': [2189, 2189], 'chainmail': [2052]}"
22425,128,"{'ring_of_aquila': [505], 'wraith_band': [430], 'javelin': [2607], 'eagle': [1980], 'phase_boots': [327], 'sobi_mask': [222], 'magic_stick': [438], 'relic': [2598], 'circlet': [430, 438], 'tpscroll': [451, 549, 748, 750, 964, 1128, 1541], 'skadi': [1657], 'quarterstaff': [2191], 'magic_wand': [505], 'ring_of_protection': [-84], 'basher': [2607], 'talisman_of_evasion': [2186], 'slippers': [430]